In [1]:
import torch
import numpy as np
from datasets import load_dataset, load_metric
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)

# load cifar10
train_ds, test_ds = load_dataset('cifar10', split=['train', 'test'])

# split trainset to train and val
splits = train_ds.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

# create label to index map
id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}

# load feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )
_val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Set the transforms
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)
test_ds.set_transform(val_transforms)

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  num_labels=10,
                                                  id2label=id2label,
                                                  label2id=label2id)


metric_name = "accuracy"
args = TrainingArguments(
    f"test-cifar-10",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

metric = load_metric(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

trainer.train()

outputs = trainer.predict(test_ds)
print(outputs.metrics)

Reusing dataset cifar10 (/home/ecbm4040/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/envs/env_pt/lib/pytho

Epoch,Training Loss,Validation Loss,Accuracy
1,0.997100,0.268149,0.981600
2,0.443100,0.160640,0.988600
3,0.310100,0.138115,0.989600


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-704
Configuration saved in test-cifar-10/checkpoint-704/config.json
Model weights saved in test-cifar-10/checkpoint-704/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-704/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-1408
Configuration saved in test-cifar-10/checkpoint-1408/config.json
Model weights saved in test-cifar-10/checkpoint-1408/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-1408/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-2112
Configuration saved in test-cifar-10/checkpoint-2112/config.json
Model weights saved in test-cifar-10/checkpoint-2112/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpo

{'test_loss': 0.14859697222709656, 'test_accuracy': 0.9839, 'test_runtime': 152.6112, 'test_samples_per_second': 65.526, 'test_steps_per_second': 1.029}
